# collections — Container datatypes
Source code: [Lib/collections/__init__.py](https://github.com/python/cpython/tree/3.8/Lib/collections/__init__.py)

## ChainMap objects

In [284]:
from collections import ChainMap, Counter, deque, defaultdict, namedtuple, OrderedDict, UserList
import random
import re, zipfile
import itertools
import csv

In [3]:
baseline = {'music': 'bach', 'art': 'rembrandt'}
adjustments = {'art': 'van gogh', 'opera': 'carmen'}
list(ChainMap(adjustments, baseline))

['music', 'art', 'opera']

In [4]:
combined = baseline.copy()
combined.update(adjustments)
list(combined)

['music', 'art', 'opera']

In [5]:
combined

{'music': 'bach', 'art': 'van gogh', 'opera': 'carmen'}

In [7]:
ChainMap(adjustments, baseline).maps

[{'art': 'van gogh', 'opera': 'carmen'}, {'music': 'bach', 'art': 'rembrandt'}]

In [9]:
ChainMap(adjustments, baseline)['art']

'van gogh'

Lookups search the underlying mappings successively until a key is found. In contrast, writes, updates, and deletions only operate on the first mapping.

In [14]:
temp = ChainMap(adjustments, baseline)
temp

ChainMap({'art': None, 'opera': 'carmen'}, {'music': 'bach', 'art': 'rembrandt'})

In [16]:
del temp['art']

In [17]:
temp

ChainMap({'opera': 'carmen'}, {'music': 'bach', 'art': 'rembrandt'})

In [19]:
adjustments

{'opera': 'carmen'}

In [18]:
temp['art']

'rembrandt'

## Counter objects

class collections.**Counter**([*iterable-or-mapping*])

A Counter is a dict subclass for counting hashable objects. It is a collection where elements are stored as dictionary keys and their counts are stored as dictionary values. Counts are allowed to be any integer value including zero or negative counts. The Counter class is similar to bags or multisets in other languages.

In [21]:
# Tally occurrences of words in a list
cnt = Counter()
for word in ['red', 'blue', 'red', 'green', 'blue', 'blue']:
    cnt[word] += 1
cnt

Counter({'red': 2, 'blue': 3, 'green': 1})

In [31]:
# Find the ten most common words in Hamlet
with zipfile.ZipFile('hamlet.zip', 'r') as myzip:
    s = myzip.read("hamlet.txt").decode().lower()
words = re.findall(r'\w+', s)
Counter(words).most_common(10)

[('the', 1091),
 ('and', 969),
 ('to', 767),
 ('of', 675),
 ('i', 633),
 ('a', 571),
 ('you', 558),
 ('my', 520),
 ('in', 451),
 ('it', 421)]

Counter objects support three methods beyond those available for all dictionaries:

**elements**()

Return an iterator over elements repeating each as many times as its count. Elements are returned in the order first encountered. If an element’s count is less than one, elements() will ignore it.

In [2]:
c = Counter(a=4, b=2, c=0, d=-2)
sorted(c.elements())

['a', 'a', 'a', 'a', 'b', 'b']

**most_common**([*n*])

Return a list of the n most common elements and their counts from the most common to the least. If n is omitted or None, most_common() returns all elements in the counter. Elements with equal counts are ordered in the order first encountered:


In [3]:
Counter('abracadabra').most_common(3)
[('a', 5), ('b', 2), ('r', 2)]

[('a', 5), ('b', 2), ('r', 2)]

**subtract**([*iterable-or-mapping*])

Elements are subtracted from an iterable or from another mapping (or counter). Like dict.update() but subtracts counts instead of replacing them. Both inputs and outputs may be zero or negative.


In [4]:
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(a=1, b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 3, 'b': 0, 'c': -3, 'd': -6})

Common patterns for working with Counter objects:

In [5]:
c = Counter('abracadabra')
c

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [6]:
sum(c.values())                 # total of all counts

11

In [8]:
c.clear()                       # reset all counts
c

Counter()

In [9]:
c = Counter('abracadabra')
list(c)                         # list unique elements

['a', 'b', 'r', 'c', 'd']

In [10]:
set(c)                          # convert to a set

{'a', 'b', 'c', 'd', 'r'}

In [11]:
dict(c)                         # convert to a regular dictionary

{'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1}

In [12]:
c.items()                       # convert to a list of (elem, cnt) pairs

dict_items([('a', 5), ('b', 2), ('r', 2), ('c', 1), ('d', 1)])

In [13]:
Counter(dict(c.items()))    # convert from a list of (elem, cnt) pairs

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [14]:
c.most_common()[:-3-1:-1]       # 3 least common elements

[('d', 1), ('c', 1), ('r', 2)]

In [17]:
c = Counter(a=4, b=2, c=0, d=-2)
c +=c                              # remove zero and negative counts // and double positive counts
c

Counter({'a': 8, 'b': 4})

Several mathematical operations are provided for combining Counter objects to produce multisets (counters that have counts greater than zero). Addition and subtraction combine counters by adding or subtracting the counts of corresponding elements. Intersection and union return the minimum and maximum of corresponding counts. Each operation can accept inputs with signed counts, but the output will exclude results with counts of zero or less.

In [18]:
c = Counter(a=3, b=1)
d = Counter(a=1, b=2)
c + d                       # add two counters together:  c[x] + d[x]

Counter({'a': 4, 'b': 3})

In [21]:
c - d                       # subtract (keeping only positive counts)

Counter({'a': 2})

In [22]:
c & d                       # intersection:  min(c[x], d[x]) 

Counter({'a': 1, 'b': 1})

In [23]:
c | d                       # union:  max(c[x], d[x])

Counter({'a': 3, 'b': 2})

Unary addition and subtraction are shortcuts for adding an empty counter or subtracting from an empty counter.

In [24]:
c = Counter(a=2, b=-4)
+c

Counter({'a': 2})

In [25]:
-c

Counter({'b': 4})

## deque objects

class collections.**deque**([*iterable[, maxlen*]])

Returns a new deque object initialized left-to-right (using append()) with data from iterable. If iterable is not specified, the new deque is empty.

Deques are a generalization of stacks and queues (the name is pronounced “deck” and is short for “double-ended queue”). Deques support thread-safe, memory efficient appends and pops from either side of the deque with approximately the same O(1) performance in either direction.

Though list objects support similar operations, they are optimized for fast fixed-length operations and incur O(n) memory movement costs for pop(0) and insert(0, v) operations which change both the size and position of the underlying data representation.

If maxlen is not specified or is None, deques may grow to an arbitrary length. Otherwise, the deque is bounded to the specified maximum length. Once a bounded length deque is full, when new items are added, a corresponding number of items are discarded from the opposite end. Bounded length deques provide functionality similar to the tail filter in Unix. They are also useful for tracking transactions and other pools of data where only the most recent activity is of interest.

Deque objects support the following methods:

**append**(*x*)

Add x to the right side of the deque.


In [27]:
d = deque(range(1, 11))
d.append(100)
d

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100])

**appendleft**(*x*)

Add x to the left side of the deque.

In [28]:
d.appendleft(200)
d

deque([200, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100])

**clear**()

Remove all elements from the deque leaving it with length 0.

In [30]:
d.clear()
d

deque([])

**copy**()

Create a shallow copy of the deque.

In [35]:
d = deque(range(1, 11))
d.append(list(range(100,500,100)))
d

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, [100, 200, 300, 400]])

In [36]:
d_copy = d.copy()
d_copy

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, [100, 200, 300, 400]])

In [37]:
d.append(500)
d_copy

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, [100, 200, 300, 400]])

In [39]:
d[10]

[100, 200, 300, 400]

In [40]:
d[10].append(1000)
d

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, [100, 200, 300, 400, 1000], 500])

In [41]:
d_copy

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, [100, 200, 300, 400, 1000]])

**count**(*x*)

Count the number of deque elements equal to x.

In [46]:
d = deque([random.randint(1, 5) for _ in  range(10)])
d

deque([4, 1, 4, 3, 3, 4, 5, 5, 5, 4])

In [47]:
d.count(3)

2

**extend**(*iterable*)

Extend the right side of the deque by appending elements from the iterable argument.

In [51]:
d = deque(range(1, 6))
d.extend(range(100, 501, 100))
d

deque([1, 2, 3, 4, 5, 100, 200, 300, 400, 500])

**extendleft**(*iterable*)

Extend the left side of the deque by appending elements from iterable. Note, the series of left appends results in reversing the order of elements in the iterable argument.


In [52]:
d = deque(range(1, 6))
d.extendleft(range(100, 501, 100))
d

deque([500, 400, 300, 200, 100, 1, 2, 3, 4, 5])

**index**(*x[, start[, stop]]*)

Return the position of x in the deque (at or after index start and before index stop). Returns the first match or raises ValueError if not found.


In [54]:
d = deque(range(1, 11))
d

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [55]:
d.index(3)

2

In [56]:
try:
    d.index(100)
except ValueError as e:
    print(repr(e))

ValueError('100 is not in deque')


**insert**(*i, x*)

Insert x into the deque at position i.
If the insertion would cause a bounded deque to grow beyond maxlen, an IndexError is raised.


In [57]:
d.insert(1, 1000)
d

deque([1, 1000, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [58]:
d = deque(range(1, 11), maxlen=10)
try:
    d.insert(1, 1000)
except IndexError as e:
    print(repr(e))

IndexError('deque already at its maximum size')


**pop**()

Remove and return an element from the right side of the deque. If no elements are present, raises an IndexError.


In [60]:
d = deque(range(1, 6))
d

deque([1, 2, 3, 4, 5])

In [61]:
d.pop()

5

In [62]:
d

deque([1, 2, 3, 4])

**popleft**()

Remove and return an element from the left side of the deque. If no elements are present, raises an IndexError.

In [63]:
d.popleft()

1

In [64]:
d

deque([2, 3, 4])

**remove**(*value*)

Remove the first occurrence of value. If not found, raises a ValueError.

In [67]:
d = deque([1,2,3,3,4,5])
d.remove(3)
d.remove(3)
d

deque([1, 2, 4, 5])

In [68]:
try:
    d.remove(3)
except ValueError as e:
    print(repr(e))

ValueError('deque.remove(x): x not in deque')


**reverse**()

Reverse the elements of the deque in-place and then return None.

In [69]:
d = deque(range(1, 6))
d

deque([1, 2, 3, 4, 5])

In [70]:
d.reverse()

In [71]:
d

deque([5, 4, 3, 2, 1])

**rotate**(*n=1*)

Rotate the deque n steps to the right. If n is negative, rotate to the left.

When the deque is not empty, rotating one step to the right is equivalent to d.appendleft(d.pop()), and rotating one step to the left is equivalent to d.append(d.popleft()).


In [72]:
d.rotate(1)

In [73]:
d

deque([1, 5, 4, 3, 2])

In [74]:
d.rotate(-2)
d

deque([4, 3, 2, 1, 5])

Deque objects also provide one read-only attribute:

**maxlen**

Maximum size of a deque or None if unbounded.


In [75]:
d = deque(range(1, 11), maxlen=10)
d.maxlen

10

In addition to the above, deques support iteration, pickling, len(d), reversed(d), copy.copy(d), copy.deepcopy(d), membership testing with the in operator, and subscript references such as d[0] to access the first element. Indexed access is O(1) at both ends but slows to O(n) in the middle. For fast random access, use lists instead.

In [77]:
d = deque(range(1, 11))
list(d)                          # list the contents of the deque

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [78]:
d[0]                             # peek at leftmost item

1

In [79]:
d[-1]                            # peek at rightmost item

10

In [80]:
list(reversed(d))                # list the contents of a deque in reverse

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [81]:
'h' in d                         # search the deque

False

In [82]:
deque(reversed(d))               # make a new deque in reverse order

deque([10, 9, 8, 7, 6, 5, 4, 3, 2, 1])

In [83]:
d.extendleft('abc')              # extendleft() reverses the input order
d

deque(['c', 'b', 'a', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

### deque Recipes
This section shows various approaches to working with deques.

Bounded length deques provide functionality similar to the tail filter in Unix:

In [115]:
def tail(zipfilename, textfilename, n=10):
    'Return the last n lines of a file'
    with zipfile.ZipFile(zipfilename, 'r') as myzip:
        with myzip.open(textfilename) as my_txt:
            lines = [line.decode().strip() for line in my_txt.readlines()]
            return deque(lines, n)

In [116]:
tail("hamlet.zip", "hamlet.txt")

deque(["The soldiers' music and the rites of war",
       'Speak loudly for him.',
       'Take up the bodies. Such a sight as this',
       'Becomes the field but here shows much amiss.',
       'Go, bid the soldiers shoot.',
       'Exeunt marching; after the which a peal of ordnance',
       'are shot off.',
       '',
       '',
       'THE END'])

Another approach to using deques is to maintain a sequence of recently added elements by appending to the right and popping to the left:

In [153]:
def moving_average(iterable, n=3):
    # moving_average([40, 30, 50, 46, 39, 44]) --> 40.0 42.0 45.0 43.0
    # http://en.wikipedia.org/wiki/Moving_average
    it = iter(iterable)
    d = deque(itertools.islice(it, n-1))
#     print(f"d: {d}")
    d.appendleft(0)
    s = sum(d)
#     print(f"s: {s}")
    for elem in it:
#         print(f"elem: {elem}")
        s += elem - d.popleft()
#         print(f"s + elem - d.popleft(): {s}")
        d.append(elem)
#         print(f"d after appending {elem}: {d}")
#         print(f"s / n: {s / n}\n")
        yield s / n

In [154]:
it = moving_average([40, 30, 50, 46, 39, 44])

In [155]:
try:
    while(av := next(it)):
        print(av, end=" ")
        pass
except StopIteration:
    pass

40.0 42.0 45.0 43.0 

A round-robin scheduler can be implemented with input iterators stored in a deque. Values are yielded from the active iterator in position zero. If that iterator is exhausted, it can be removed with popleft(); otherwise, it can be cycled back to the end with the rotate() method:

In [156]:
def roundrobin(*iterables):
    "roundrobin('ABC', 'D', 'EF') --> A D E B F C"
    iterators = deque(map(iter, iterables))
    while iterators:
        try:
            while True:
                yield next(iterators[0])
                iterators.rotate(-1)
        except StopIteration:
            # Remove an exhausted iterator.
            iterators.popleft()

In [157]:
it = roundrobin('ABC', 'D', 'EF')
try:
    while(next_val := next(it)):
        print(next_val, end=" ")
        pass
except StopIteration:
    pass

A D E B F C 

The rotate() method provides a way to implement deque slicing and deletion. For example, a pure Python implementation of del d[n] relies on the rotate() method to position elements to be popped:

In [158]:
def delete_nth(d, n):
    d.rotate(-n)
    d.popleft()
    d.rotate(n)

In [165]:
d = deque(range(1, 11))
d

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [166]:
try:
    delete_nth(d, d.index(4)) # delete 4
except ValueError as e:
    print(repr(e))
d

deque([1, 2, 3, 5, 6, 7, 8, 9, 10])

To implement deque slicing, use a similar approach applying rotate() to bring a target element to the left side of the deque. Remove old entries with popleft(), add new entries with extend(), and then reverse the rotation. With minor variations on that approach, it is easy to implement Forth style stack manipulations such as dup, drop, swap, over, pick, rot, and roll.

## defaultdict objects

class collections.**defaultdict**([*default_factory[, ...]*])

Returns a new dictionary-like object. defaultdict is a subclass of the built-in dict class. It overrides one method and adds one writable instance variable. The remaining functionality is the same as for the dict class and is not documented here.

The first argument provides the initial value for the default_factory attribute; it defaults to None. All remaining arguments are treated the same as if they were passed to the dict constructor, including keyword arguments.

defaultdict objects support the following method in addition to the standard dict operations:

**\_\_missing\_\_**(*key*)
If the default_factory attribute is None, this raises a KeyError exception with the key as argument.

If default_factory is not None, it is called without arguments to provide a default value for the given key, this value is inserted in the dictionary for the key, and returned.

If calling default_factory raises an exception this exception is propagated unchanged.

This method is called by the __getitem__() method of the dict class when the requested key is not found; whatever it returns or raises is then returned or raised by __getitem__().

Note that __missing__() is not called for any operations besides __getitem__(). This means that get() will, like normal dictionaries, return None as a default rather than using default_factory.

defaultdict objects support the following instance variable:

**default_factory**

This attribute is used by the __missing__() method; it is initialized from the first argument to the constructor, if present, or to None, if absent.

Using list as the default_factory, it is easy to group a sequence of key-value pairs into a dictionary of lists:

In [170]:
s = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
d = defaultdict(list)
for k, v in s:
    d[k].append(v)
d

defaultdict(list, {'yellow': [1, 3], 'blue': [2, 4], 'red': [1]})

In [171]:
sorted(d.items())

[('blue', [2, 4]), ('red', [1]), ('yellow', [1, 3])]

When each key is encountered for the first time, it is not already in the mapping; so an entry is automatically created using the default_factory function which returns an empty list. The list.append() operation then attaches the value to the new list. When keys are encountered again, the look-up proceeds normally (returning the list for that key) and the list.append() operation adds another value to the list. This technique is simpler and faster than an equivalent technique using dict.setdefault():

In [172]:
d = {}
for k, v in s:
    d.setdefault(k, []).append(v)
sorted(d.items())

[('blue', [2, 4]), ('red', [1]), ('yellow', [1, 3])]

Setting the *default_factory* to *int* makes the defaultdict useful for counting (like a bag or multiset in other languages):

In [173]:
s = 'mississippi'
d = defaultdict(int)
for k in s:
    d[k] += 1
sorted(d.items())

[('i', 4), ('m', 1), ('p', 2), ('s', 4)]

In [177]:
# The Counter is simpler though
c = Counter(s)
c

Counter({'m': 1, 'i': 4, 's': 4, 'p': 2})

When a letter is first encountered, it is missing from the mapping, so the default_factory function calls int() to supply a default count of zero. The increment operation then builds up the count for each letter.

The function int() which always returns zero is just a special case of constant functions. A faster and more flexible way to create constant functions is to use a lambda function which can supply any constant value (not just zero):


In [187]:
def constant_factory(value):
    return lambda: value

d = defaultdict(constant_factory('Mary'))
d.update(name='John', action='ran')
print(f"{d['name']} {d['action']} to {d['object']}")

John ran to Mary


In [188]:
d

defaultdict(<function __main__.constant_factory.<locals>.<lambda>()>,
            {'name': 'John', 'action': 'ran', 'object': 'Mary'})

Setting the *default_factory* to set makes the defaultdict useful for building a dictionary of sets:

In [190]:
s = [('red', 1), ('blue', 2), ('red', 3), ('blue', 4), ('red', 1), ('blue', 4)]
d = defaultdict(set)

for k, v in s:
    d[k].add(v)
    
sorted(d.items())

[('blue', {2, 4}), ('red', {1, 3})]

## namedtuple() Factory Function for Tuples with Named Fields

Named tuples assign meaning to each position in a tuple and allow for more readable, self-documenting code. They can be used wherever regular tuples are used, and they add the ability to access fields by name instead of position index.

collections.**namedtuple**(typename, field_names, *, rename=False, defaults=None, module=None)

Returns a new tuple subclass named typename. The new subclass is used to create tuple-like objects that have fields accessible by attribute lookup as well as being indexable and iterable. Instances of the subclass also have a helpful docstring (with typename and field_names) and a helpful __repr__() method which lists the tuple contents in a name=value format.

The field_names are a sequence of strings such as ['x', 'y']. Alternatively, field_names can be a single string with each fieldname separated by whitespace and/or commas, for example 'x y' or 'x, y'.

Any valid Python identifier may be used for a fieldname except for names starting with an underscore. Valid identifiers consist of letters, digits, and underscores but do not start with a digit or underscore and cannot be a keyword such as class, for, return, global, pass, or raise.

If rename is true, invalid fieldnames are automatically replaced with positional names. For example, ['abc', 'def', 'ghi', 'abc'] is converted to ['abc', '_1', 'ghi', '_3'], eliminating the keyword def and the duplicate fieldname abc.

defaults can be None or an iterable of default values. Since fields with a default value must come after any fields without a default, the defaults are applied to the rightmost parameters. For example, if the fieldnames are ['x', 'y', 'z'] and the defaults are (1, 2), then x will be a required argument, y will default to 1, and z will default to 2.

If module is defined, the __module__ attribute of the named tuple is set to that value.

Named tuple instances do not have per-instance dictionaries, so they are lightweight and require no more memory than regular tuples.


In [196]:
# Basic example
Point = namedtuple('Point', ['x', 'y'])
p = Point(11, y=22)     # instantiate with positional or keyword arguments
p[0] + p[1]             # indexable like the plain tuple (11, 22)

33

In [197]:
x, y = p                # unpack like a regular tuple
x, y

(11, 22)

In [198]:
p.x + p.y               # fields also accessible by name

33

In [199]:
p                       # readable __repr__ with a name=value style

Point(x=11, y=22)

Named tuples are especially useful for assigning field names to result tuples returned by the csv or sqlite3 modules:

In [212]:
EmployeeRecord = namedtuple('EmployeeRecord', 'name, age, title, department, paygrade')

for emp in map(EmployeeRecord._make, csv.reader(open("employees.csv", "r"), delimiter=";")):
    print(f"{emp.name},\t{emp.title}")

# import sqlite3
# conn = sqlite3.connect('/companydata')
# cursor = conn.cursor()
# cursor.execute('SELECT name, age, title, department, paygrade FROM employees')
# for emp in map(EmployeeRecord._make, cursor.fetchall()):
#     print(emp.name, emp.title)

п»їemployee_1,	title_5
employee_2,	title_5
employee_3,	title_5
employee_4,	title_3
employee_5,	title_5
employee_6,	title_2
employee_7,	title_1
employee_8,	title_3
employee_9,	title_1
employee_10,	title_5


In addition to the methods inherited from tuples, named tuples support three additional methods and two attributes. To prevent conflicts with field names, the method and attribute names start with an underscore.

classmethod somenamedtuple.**_make**(*iterable*)

Class method that makes a new instance from an existing sequence or iterable.


In [214]:
t = [11, 22]
Point._make(t)

Point(x=11, y=22)

somenamedtuple.**_asdict**()

Return a new dict which maps field names to their corresponding values:

In [215]:
p = Point(x=11, y=22)
p._asdict()

{'x': 11, 'y': 22}

somenamedtuple.**_replace**(\*\*kwargs)

Return a new instance of the named tuple replacing specified fields with new values:

In [216]:
p = Point(x=11, y=22)
p._replace(x=33)

Point(x=33, y=22)

somenamedtuple.**_fields**

Tuple of strings listing the field names. Useful for introspection and for creating new named tuple types from existing named tuples.

In [217]:
p._fields            # view the field names

('x', 'y')

In [218]:
Color = namedtuple('Color', 'red green blue')
Pixel = namedtuple('Pixel', Point._fields + Color._fields)
Pixel(11, 22, 128, 255, 0)

Pixel(x=11, y=22, red=128, green=255, blue=0)

somenamedtuple.**_field_defaults**

Dictionary mapping field names to default values.

In [225]:
Account = namedtuple('Account', ['type', 'balance'], defaults=[0])
Account._field_defaults

{'balance': 0}

In [226]:
Account('premium')

Account(type='premium', balance=0)

To retrieve a field whose name is stored in a string, use the getattr() function:

In [227]:
getattr(p, 'x')

11

To convert a dictionary to a named tuple, use the double-star-operator (as described in Unpacking Argument Lists):

In [229]:
d = {'x': 11, 'y': 22}
Point(**d)

Point(x=11, y=22)

Since a named tuple is a regular Python class, it is easy to add or change functionality with a subclass. Here is how to add a calculated field and a fixed-width print format:

In [230]:
class Point(namedtuple('Point', ['x', 'y'])):
    __slots__ = ()
    @property
    def hypot(self):
        return (self.x ** 2 + self.y ** 2) ** 0.5
    def __str__(self):
        return 'Point: x=%6.3f  y=%6.3f  hypot=%6.3f' % (self.x, self.y, self.hypot)

In [231]:
for p in Point(3, 4), Point(14, 5/7):
    print(p)

Point: x= 3.000  y= 4.000  hypot= 5.000
Point: x=14.000  y= 0.714  hypot=14.018


In [232]:
p1 = Point(3, 4)

In [239]:
print(f"p1.x: {p1.x},\np1.y: {p1.y}\np1.hypot: {p1.hypot}")

p1.x: 3,
p1.y: 4
p1.hypot: 5.0


The subclass shown above sets *\_\_slots\_\_* to an empty tuple. This helps keep memory requirements low by preventing the creation of instance dictionaries.

Subclassing is not useful for adding new, stored fields. Instead, simply create a new named tuple type from the *_fields* attribute:

In [240]:
Point3D = namedtuple('Point3D', Point._fields + ('z',))

In [241]:
Point3D(11,12,13)

Point3D(x=11, y=12, z=13)

Docstrings can be customized by making direct assignments to the *\_\_doc\_\_* fields:

In [242]:
Book = namedtuple('Book', ['id', 'title', 'authors'])
Book.__doc__ += ': Hardcover book in active collection'
Book.id.__doc__ = '13-digit ISBN'
Book.title.__doc__ = 'Title of first printing'
Book.authors.__doc__ = 'List of authors sorted by last name'

In [250]:
Book.__doc__

'Book(id, title, authors): Hardcover book in active collection'

In [251]:
Book.id.__doc__

'13-digit ISBN'

## OrderedDict objects

Ordered dictionaries are just like regular dictionaries but have some extra capabilities relating to ordering operations. They have become less important now that the built-in dict class gained the ability to remember insertion order (this new behavior became guaranteed in Python 3.7).

Some differences from dict still remain:
•The regular dict was designed to be very good at mapping operations. Tracking insertion order was secondary.
•The OrderedDict was designed to be good at reordering operations. Space efficiency, iteration speed, and the performance of update operations were secondary.
•Algorithmically, OrderedDict can handle frequent reordering operations better than dict. This makes it suitable for tracking recent accesses (for example in an LRU cache).
•The equality operation for OrderedDict checks for matching order.
•The popitem() method of OrderedDict has a different signature. It accepts an optional argument to specify which item is popped.
•OrderedDict has a move_to_end() method to efficiently reposition an element to an endpoint.
•Until Python 3.8, dict lacked a __reversed__() method.

class collections.**OrderedDict**([*items*])

Return an instance of a dict subclass that has methods specialized for rearranging dictionary order.


**popitem**(*last=True*)

The popitem() method for ordered dictionaries returns and removes a (key, value) pair. The pairs are returned in LIFO order if last is true or FIFO order if false.

**move_to_end**(*key, last=True*)

Move an existing key to either end of an ordered dictionary. The item is moved to the right end if last is true (the default) or to the beginning if last is false. Raises KeyError if the key does not exist:

In [255]:
d = OrderedDict.fromkeys('abcde')
d.move_to_end('b')
''.join(d.keys())

'acdeb'

In [256]:
d.move_to_end('b', last=False)
''.join(d.keys())

'bacde'

In [264]:
for k, i in zip(d.keys(), range(len(d))):
    d[k] = i+1
for k, v in d.items():
    print(f"{k}: {v}")

b: 1
a: 2
c: 3
d: 4
e: 5


In addition to the usual mapping methods, ordered dictionaries also support reverse iteration using reversed().

In [270]:
for k in reversed(d):
    print(f"{k}: {d[k]}")

e: 5
d: 4
c: 3
a: 2
b: 1


Equality tests between OrderedDict objects are order-sensitive and are implemented as list(od1.items())==list(od2.items()). Equality tests between OrderedDict objects and other Mapping objects are order-insensitive like regular dictionaries. This allows OrderedDict objects to be substituted anywhere a regular dictionary is used.

In [280]:
d = OrderedDict.fromkeys('abcde')
d

OrderedDict([('a', None), ('b', None), ('c', None), ('d', None), ('e', None)])

In [281]:
dict(OrderedDict.fromkeys(reversed(d)))

{'e': None, 'd': None, 'c': None, 'b': None, 'a': None}

In [282]:
d == reversed(d)

False

In [283]:
d == dict(OrderedDict.fromkeys(reversed(d)))

True

It is straightforward to create an ordered dictionary variant that remembers the order the keys were last inserted. If a new entry overwrites an existing entry, the original insertion position is changed and moved to the end:

## UserList objects
This class acts as a wrapper around list objects. It is a useful base class for your own list-like classes which can inherit from them and override existing methods or add new ones. In this way, one can add new behaviors to lists.

The need for this class has been partially supplanted by the ability to subclass directly from list; however, this class can be easier to work with because the underlying list is accessible as an attribute.

class collections.**UserList**([*list*])

Class that simulates a list. The instance’s contents are kept in a regular list, which is accessible via the data attribute of UserList instances. The instance’s contents are initially set to a copy of list, defaulting to the empty list []. list can be any iterable, for example a real Python list or a UserList object.

In addition to supporting the methods and operations of mutable sequences, UserList instances provide the following attribute:

**data**

A real list object used to store the contents of the UserList class.

Subclassing requirements: Subclasses of UserList are expected to offer a constructor which can be called with either no arguments or one argument. List operations which return a new sequence attempt to create an instance of the actual implementation class. To do so, it assumes that the constructor can be called with a single parameter, which is a sequence object used as a data source.

If a derived class does not wish to comply with this requirement, all of the special methods supported by this class will need to be overridden; please consult the sources for information about the methods which need to be provided in that case.


In [285]:
ul = UserList([])
ul

[]

In [286]:
ul = ul + [1,2,3]
ul

[1, 2, 3]

In [287]:
ul.data

[1, 2, 3]

In [323]:
class MyList(UserList):   
    def __pow__(self, b):
        """Double self.data b times."""
        for i in range(b):
            self.data += self.data
#             print(f"i: {i}, self.data: {self.data}")

In [324]:
ml = MyList([1,2,3])
ml

[1, 2, 3]

In [325]:
ml**3

In [326]:
ml

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

## UserString objects
The class, UserString acts as a wrapper around string objects. The need for this class has been partially supplanted by the ability to subclass directly from str; however, this class can be easier to work with because the underlying string is accessible as an attribute.

class collections.**UserString**(*seq*)

Class that simulates a string object. The instance’s content is kept in a regular string object, which is accessible via the data attribute of UserString instances. The instance’s contents are initially set to a copy of seq. The seq argument can be any object which can be converted into a string using the built-in str() function.

In addition to supporting the methods and operations of strings, UserString instances provide the following attribute:

**data**

A real str object used to store the contents of the UserString class.
